In [1]:
import os
import numpy as np
import cv2 as cv
from PIL import Image
from tensorflow.keras.utils import normalize
from keras.models import load_model
import matplotlib.pyplot as plt

SIZE = 256

disc_unet = load_model('D:/Work/Retinopathy/Sample project/glaucoma/using unet/models/retrain_full_noise_t1_ha_disc.h5')
cup_unet = load_model('D:/Work/Retinopathy/Sample project/glaucoma/using unet/models/retrain_full_noise_t1_ha_cup.h5')


In [15]:
def masking(img_array, img_num = 0, img_side_size = SIZE):
    
    pred = img_array[img_num]
    pred[pred>0.5] = 1
    pred[pred<0.5] = 0
    mask = pred.reshape(img_side_size, img_side_size)
    plt.imsave('pred_mask2.png',mask, cmap = 'gray')
    mask =cv.imread('pred_mask2.png')
    os.remove("pred_mask2.png")

    return mask 

In [3]:
def preprocess(image):
    image_dataset = []
    image = cv.imread(image,0)
    image = Image.fromarray(image)
    image = image.resize((SIZE,SIZE))
    image_dataset.append(np.array(image))
    image_dataset = np.expand_dims(normalize(np.array(image_dataset), axis = 1),3)
    return image_dataset

In [4]:
def segmentation(model,image):
    image = preprocess(image)
    prediction = model.predict(image)
    prediction = masking(prediction)
    return prediction

In [5]:
def contours(image):
    #_,th = cv.threshold(image, 200, 255, cv.THRESH_BINARY)
    edges = cv.Canny(image, 30, 200)
    contours,_ = cv.findContours(edges, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)
    largest = max(contours, key= cv.contourArea)
    
    return largest

In [6]:
def diameter(image):
    image = contours(image)
    ellipse = cv.fitEllipse(image)
    (xc, yc),(d1, d2), angle = ellipse 
    diameter = int(max(d1,d2))
    return diameter
    

In [12]:
def glaucoma(path):
    
    disc = segmentation(disc_unet,path)
    cup = segmentation(cup_unet,path)

    cup_diameter = diameter(cup)
    disc_diameter = diameter(disc)    

    


    cdr = cup_diameter/disc_diameter
    print(cdr) 

    if cdr >= 0.8:
        diagnosis = 'Severe'
        print('Risk of Glaucoma is ',diagnosis)

    elif cdr >=0.5 and cdr <0.8:
                        
        diagnosis = 'Moderate'
        print('Risk of Glaucoma is ',diagnosis)
        
                    
    elif cdr >0.3 and cdr <0.5:
        diagnosis = 'Mild'
        print('Risk of Glaucoma is ',diagnosis)

    else:
        diagnosis = 'None'
        print('Risk of Glaucoma is ',diagnosis)

    

In [21]:
path = 'D:/Work/Retinopathy/sample project/glaucoma/using unet/test/left_g-1-L.png'
#path = 'test/01_g.jpg'
glaucoma(path)

0.5652173913043478
Risk of Glaucoma is  Moderate


In [14]:
#disc = segmentation(disc_unet,path)
cup = segmentation(cup_unet,path)